In [ ]:
import pandas as pd
import cleanup_utils

In [ ]:
ROOT_PATH = '/data/NLP/tacred/data'

In [ ]:
def prepare_df(df) -> pd.DataFrame:
    df = (
        df
        .assign(subjectSpan=df.apply(lambda x: [x['subj_start'], x['subj_end']], axis=1))
        .assign(objectSpan=df.apply(lambda x: [x['obj_start'], x['obj_end']], axis=1))
        .loc[:,['relation','objectSpan','subjectSpan','subj_type','obj_type','token','stanford_ner']]
        .rename(columns={'stanford_ner': 'labels'})
        .rename(columns={'token': 'tokens'})
    )
    df = (
        df
        .assign(tokens=df['tokens'].apply(cleanup_utils.normalize_glove))
        .assign(labels=df['labels'].apply(cleanup_utils.process_labels))
        .assign(object=df.apply(lambda row: ' '.join(row['tokens'][row['objectSpan'][0]:row['objectSpan'][1]+1]), axis=1))
        .assign(subject=df.apply(lambda row: ' '.join(row['tokens'][row['subjectSpan'][0]:row['subjectSpan'][1]+1]), axis=1))
    )

    return df


for _dir in os.listdir(f'{ROOT_PATH}/rev/'):
    prepare_df(pd.read_json(f"{ROOT_PATH}/rev/{_dir}").reset_index(drop=True)).to_json(f'{ROOT_PATH}/preprocessed/{_dir}')


In [ ]:
train_df = pd.read_json(f"{ROOT_PATH}/preprocessed/train.json").reset_index(drop=True)
dev_df = pd.read_json(f"{ROOT_PATH}/preprocessed/dev.json").reset_index(drop=True)
test_df = pd.read_json(f"{ROOT_PATH}/preprocessed/test.json").reset_index(drop=True)

In [ ]:
dev_df.head(10)

In [ ]:
print(pd.unique(train_df['relation']))

In [ ]:
print(pd.unique(train_df.explode('labels')['labels']))